<a href="https://colab.research.google.com/github/keshav1441/Amazon_ML/blob/main/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd

# Load each CSV file into a DataFrame
sample_test_df = pd.read_csv('sample_data/sample_test.csv')
sample_test_out_df = pd.read_csv('sample_data/sample_test_out.csv')
sample_test_out_fail_df = pd.read_csv('sample_data/sample_test_out_fail.csv')
test_df = pd.read_csv('sample_data/test.csv')
train_df = pd.read_csv('sample_data/train.csv')

# Display the first few rows of each DataFrame to verify
print(sample_test_df.head())
print(sample_test_out_df.head())
print(sample_test_out_fail_df.head())
print(test_df.head())
print(train_df.head())


   index                                         image_link  group_id  \
0      0  https://m.media-amazon.com/images/I/41-NCxNuBx...    658003   
1      1  https://m.media-amazon.com/images/I/41-NCxNuBx...    658003   
2      2  https://m.media-amazon.com/images/I/417NJrPEk+...    939426   
3      3  https://m.media-amazon.com/images/I/417SThj+Sr...    276700   
4      4  https://m.media-amazon.com/images/I/417SThj+Sr...    276700   

                     entity_name  
0                          width  
1                          depth  
2  maximum_weight_recommendation  
3                        voltage  
4                        wattage  
   index        prediction
0      0         21.9 foot
1      1           10 foot
2      2               NaN
3      3   289.52 kilovolt
4      4  1078.99 kilowatt
   index        prediction
0      0         21.9 foot
1      1           10 foot
2      2               NaN
3      3   289.52 kilovolt
4      4  1078.99 kilowatt
   index                   

In [ ]:
# Get the total count of each unique entity_name
entity_name_counts = train_df['entity_name'].value_counts()

# Print the total counts for each entity name
print(entity_name_counts)


entity_name
item_weight                      102786
depth                             45127
width                             44183
height                            43597
voltage                            9466
wattage                            7755
item_volume                        7682
maximum_weight_recommendation      3263
Name: count, dtype: int64


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Sample 5% of the dataset
train_df = train_df.sample(frac=0.04, random_state=42).reset_index(drop=True)

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit and transform the labels to indices
train_df['entity_value_index'] = label_encoder.fit_transform(train_df['entity_name'])

# Save the sampled and updated dataset
train_df.to_csv('sample_data/train_df.csv', index=False)

# Display the first few rows to confirm the new column
print(train_df.head())


                                          image_link  group_id  entity_name  \
0  https://m.media-amazon.com/images/I/81nEfmLyyM...    630390  item_weight   
1  https://m.media-amazon.com/images/I/51gz4v5BxT...    751532        width   
2  https://m.media-amazon.com/images/I/61D2A-LxNn...    478233       height   
3  https://m.media-amazon.com/images/I/51INxCAF0T...    943839        depth   
4  https://m.media-amazon.com/images/I/81-zmM+W8B...    487566  item_weight   

   entity_value  entity_value_index  
0  1.0 kilogram                   3  
1     78.0 inch                   7  
2     10.1 inch                   1  
3     2.88 inch                   0  
4     1.8 pound                   3  


In [ ]:
from transformers import ViltFeatureExtractor
model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViltFeatureExtractor.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vilt/feature_extraction_vilt.py:28: FutureWarning: The class ViltFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViltImageProcessor instead.
  warnings.warn(


In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO
import torch
import torch.nn as nn
import torch.optim as optim



In [ ]:
# Load the pre-trained CLIP model and processor (ViT-B/16 for accuracy)
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")



config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Define a fine-tuned model class for entity_value prediction using CLIP features
class CLIPFineTunedModel(nn.Module):
    def __init__(self, clip_model, output_dim):
        super(CLIPFineTunedModel, self).__init__()
        self.clip_model = clip_model
        # Fully connected layer for entity_value classification
        self.fc = nn.Linear(clip_model.config.projection_dim, output_dim)

    def forward(self, inputs):
        # Extract image and text embeddings from CLIP model
        with torch.no_grad():
            outputs = self.clip_model(**inputs)

        # Average image and text embeddings (other fusion strategies could be used)
        combined_features = (outputs.image_embeds + outputs.text_embeds) / 2

        # Pass combined features through the classification layer to predict entity_value
        return self.fc(combined_features)


In [ ]:

# Initialize the fine-tuned model for the number of unique entity_value classes
output_dim = len(train_df['entity_value'].unique())  # Number of entity_value categories
fine_tuned_model = CLIPFineTunedModel(model, output_dim)
fine_tuned_model.train()  # Set the model to training mode

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()  # Assuming entity_value is a categorical target
optimizer = optim.Adam(fine_tuned_model.parameters(), lr=1e-4)


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit and transform the labels to indices
train_df['entity_value_index'] = label_encoder.fit_transform(train_df['entity_name'])

# Save the updated dataset if needed
train_df.to_csv('sample_data/train_df.csv', index=False)

# Display the first few rows to confirm the new column
print(train_df.head())


                                          image_link  group_id  entity_name  \
0  https://m.media-amazon.com/images/I/61I9XdN6OF...    748919  item_weight   
1  https://m.media-amazon.com/images/I/71gSRbyXmo...    916768  item_volume   
2  https://m.media-amazon.com/images/I/61BZ4zrjZX...    459516  item_weight   
3  https://m.media-amazon.com/images/I/612mrlqiI4...    459516  item_weight   
4  https://m.media-amazon.com/images/I/617Tl40LOX...    731432  item_weight   

     entity_value  entity_value_index  
0      500.0 gram                   3  
1         1.0 cup                   2  
2      0.709 gram                   3  
3      0.709 gram                   3  
4  1400 milligram                   3  


In [ ]:
# Get the total count of each unique entity_name
entity_name_counts = train_df['entity_name'].value_counts()

# Print the total counts for each entity name
print(entity_name_counts)


entity_name
item_weight                      4111
depth                            1809
height                           1784
width                            1752
voltage                           370
wattage                           294
item_volume                       286
maximum_weight_recommendation     148
Name: count, dtype: int64


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
fine_tuned_model = CLIPFineTunedModel(model, output_dim).to(device)


In [ ]:
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import torch

# Example training loop
num_epochs = 5  # Define the number of epochs for training
for epoch in range(num_epochs):
    for idx, row in train_df.iterrows():
        image_url = row['image_link']  # Extract the image URL

        try:
            # Load image from the URL in the dataset
            response = requests.get(image_url)
            response.raise_for_status()  # Check if the request was successful

            try:
                img = Image.open(BytesIO(response.content)).convert('RGB')
            except (OSError, UnidentifiedImageError) as e:
                print(f"Error opening image at row {idx}, URL: {image_url}. Error: {e}")
                continue  # Skip this image and continue with the next
        except requests.RequestException as e:
            print(f"Error fetching image from URL: {image_url}. Error: {e}")
            continue  # Skip this image and continue with the next

        try:
            # Prepare inputs using CLIP processor
            inputs = processor(text=row['entity_name'], images=img, return_tensors="pt", padding=True)

            # Move inputs to GPU
            inputs = {k: v.to(device) for k, v in inputs.items()}

            # Forward pass: predict entity_value
            outputs = fine_tuned_model(inputs)

            # Convert entity_value to index for loss computation
            target = torch.tensor([row['entity_value_index']], dtype=torch.long).to(device)
            loss = criterion(outputs, target)

            # Backward pass and optimization step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Log the training progress
            if idx % 100 == 0:
                print(f'Epoch {epoch + 1}/{num_epochs}, Batch {idx}, Loss: {loss.item()}')

        except Exception as e:
            print(f"Error during training at row {idx}. Error: {e}")
            continue  # Skip this row and continue with the next

# Save the fine-tuned model's state
torch.save(fine_tuned_model.state_dict(), "clip_finetuned_model.pth")
